In [1]:
# web scraping
from bs4 import BeautifulSoup

# date parser
from datetime import datetime
from dateutil.parser import parse

# web driver
import undetected_chromedriver as uc 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# random wait
import time
import random

# mongodb connection
from pymongo import MongoClient

# progress bar
from tqdm import tqdm

# saving to excel
import pandas as pd

import requests
from io import BytesIO

In [2]:
# date parser function
def parse_date(date_str):
    date1 = datetime.strptime(date_str, '%d-%b-%Y')
    output_date_string = date1.strftime("%Y-%m-%d")
    return output_date_string

In [3]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

In [4]:
# reading all BL number from google sheet
r = requests.get('https://docs.google.com/spreadsheets/d/e/2PACX-1vTAFNFtEXE7yWMYbkJF81On78NhnYpQisqdDbhvW4aPjPYuSMLVSrIDuIK5D6zY0cS9kboGaFazqL3e/pub?output=csv')
data = r.content
df = pd.read_csv(BytesIO(data))
liners_bl = df[df["LINERS"] == "CNC"]
bl_list = list(set(liners_bl["BL Number"]))

In [5]:
# view bl list
len(bl_list)

30

In [6]:
# collecting failed BL to track
gagal = []

<h3><strong>Web Scraping Flow</strong></h3>
<ol>
 <li>Acquiring every BL Number to track and store it into a list of BL</li>
 <li>Iterate all of the list and search it through the liners web</li>
 <li>Take list of container number and store it into a list</li>
 <li>If container number>1 iterate through all container for container's milestone info and store it into a dictionary and append that to a list of dict</li>
 <li>If container = 1 take container info and store it into a dictionary and append it to list of dictionary</li>
 <li>Modify dictionaries to match db's column template</li>
 <li>Insert list of dictionary to mongo db or export it into and excel file</li>
</ol>

In [7]:
hasil_akhir = []

# web scripting
options = Options()
options.add_argument("--window-size=1920,1280")
driver = uc.Chrome()
driver.get("https://www.cnc-line.com/ebusiness/tracking/search")

# search box
search_box = driver.find_element(By. XPATH, '/html/body/div[2]/main/section/div/div/form[3]/fieldset/div/div[1]/span[1]/input[2]')
search_box.send_keys(bl_list[random.randrange(0,len(bl_list))])

# click search
time.sleep(1)
search_button = driver.find_element(By.XPATH, '/html/body/div[2]/main/section/div/div/form[3]/fieldset/div/div[2]/button')
time.sleep(1)
search_button.click()

for q, bls in enumerate(tqdm(bl_list)):
    try:
        time.sleep(random.randrange(2,6))
        # masukin BL baru
        search_box2 = driver.find_element(By. XPATH, '/html/body/div[2]/main/section[1]/div/div/form[3]/fieldset/div/div[1]/span[1]/input[2]')
        # driver.execute_script("window.scrollTo(100,document.body.scrollHeight);")
        search_box2.clear()
        search_box2.send_keys(bls)
        time.sleep(1.1)
        search_button = driver.find_element(By.XPATH, '/html/body/div[2]/main/section[1]/div/div/form[3]/fieldset/div/div[2]/button')
        time.sleep(1)
        search_button.click()
    
        time.sleep(random.randrange(3,7))
        # taking data from web 
        soup = BeautifulSoup(driver.page_source, 'lxml')

        # finding containers in bl
        containers_in_bl = soup.find_all('dl',{'class','container-ref'})
        list_of_containers = []
        for tag in containers_in_bl:
            for e, f  in enumerate(tag.find_all('span')):
                if len(f.text) == 11:
                    list_of_containers.append(f.text)

        # web scraping flow if bl have >1 container number
        if len(list_of_containers)>1:

            print(bls, 'consist of ', len(list_of_containers),' containers' )
            # clicking more button
            more_button= WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/main/section[2]/div/div/ul/li[1]/article/section[2]/div[1]/div/label'))) 
            more_button.click()

            time.sleep(3)
            soup1 = BeautifulSoup(driver.page_source, 'lxml')

            # finding milestone and expected data
            data_milestone = soup1.find_all('tr',{'class','k-master-row done'}) + soup1.find_all('tr',{'class','k-alt k-master-row done'}) + soup1.find_all('tr',{'class','k-alt k-master-row current'}) + soup1.find_all('tr',{'class','k-master-row current'})
            data_expected = soup1.find_all('tr',{'class','k-master-row inactivek-alt'}) + soup1.find_all('tr',{'class','k-master-row inactive'})
            headline = soup1.find_all('strong')

            # making a list of milestones
            list_of_milestone_date = []
            list_of_milestone_movement = []
            list_of_milestone_cities = []
            for tag in data_milestone:
                for a, b  in enumerate(tag.find_all('span',{'class','calendar'})):
                    list_of_milestone_date.append(b.text[-11:])
                for c, d  in enumerate(tag.find_all('span',{'class','capsule'})):
                    list_of_milestone_movement.append(d.text)
                for e, f  in enumerate(tag.find_all('div',{'class','location row js-bubble'})):
                    list_of_milestone_cities.append(f.text[:-16])

            # making a list of expected dict
            list_of_expected_date = []
            list_of_expected_movement = []
            list_of_expected_cities = []
            for tag in data_expected:
                for g, h  in enumerate(tag.find_all('span',{'class','calendar'})):
                    list_of_expected_date.append(h.text[-11:])
                for i, j  in enumerate(tag.find_all('span',{'class','capsule'})):
                    list_of_expected_movement.append(j.text)
                for k, l  in enumerate(tag.find_all('div',{'class','location row js-bubble'})):
                    list_of_expected_cities.append(l.text[:-16])

            # Apending data from list to current dict
            current_dict = {}
            for m, milestone in enumerate(sorted(list_of_milestone_date)):
                if list_of_milestone_cities[m] == headline[1].text[:-5] or list_of_milestone_cities[m] == headline[2].text[:-5]:
                    case_milestone = {list_of_milestone_movement[m] +' '+ list_of_milestone_cities[m] :list_of_milestone_date[m]}
                    current_dict.update(case_milestone)

            for e, expected in enumerate(sorted(list_of_expected_date)):
                if list_of_expected_cities[e] == headline[2].text[:-5] or list_of_expected_cities[e] == headline[1].text[:-5]:
                    case_expected = {"EXPECTED " + list_of_expected_movement[e] + ' ' +  list_of_expected_cities[e]:list_of_expected_date[e]}
                    current_dict.update(case_expected)

            # sorting dictionary using date
            current_dict = dict(sorted(current_dict.items(), key=lambda item: parse_date(item[1])))

            # cities from to
            cities = soup1.find_all('ul',{'class','timeline--items'})

            for x, container in enumerate(list_of_containers):
                appending_dict = current_dict.copy()
                for tag in cities:
                    for cc, city in enumerate(tag.find_all('div',{'class':'timeline--item-description'})):
                        case_city = {city.text.replace("\n","")[:3]:city.text.replace("\n","")[3:-5]}
                        appending_dict.update(case_city)
                appending_dict.update({"Container Number":container})
                appending_dict.update({"BL Number":bls})
                appending_dict.update({"Liners":"CNC"})

                # appending to list of dict
                hasil_akhir.append(appending_dict)
                print(bls, container, " DONE")
            time.sleep(1)
        else:
            print(bls, 'consist of 1 containers' )
            time.sleep(1)
            soup2 = BeautifulSoup(driver.page_source, 'lxml')

            current_dict = {}

            # finding milestone and expected data
            data_milestone = soup2.find_all('tr',{'class','k-master-row done'}) + soup2.find_all('tr',{'class','k-alt k-master-row done'}) + soup2.find_all('tr',{'class','k-alt k-master-row current'}) + soup2.find_all('tr',{'class','k-master-row current'})
            data_expected = soup2.find_all('tr',{'class','k-alt k-master-row inactive'}) + soup2.find_all('tr',{'class','k-master-row inactive'})
            headline = soup2.find_all('strong')

            # making a list of milestones
            list_of_milestone_date = []
            list_of_milestone_movement = []
            list_of_milestone_cities = []
            for tag in data_milestone:
                for a, b  in enumerate(tag.find_all('span',{'class','calendar'})):
                    list_of_milestone_date.append(b.text[-11:])
                for c, d  in enumerate(tag.find_all('span',{'class','capsule'})):
                    list_of_milestone_movement.append(d.text)
                for e, f  in enumerate(tag.find_all('div',{'class','location row js-bubble'})):
                    list_of_milestone_cities.append(f.text[:-16])

            # making a list od expected dict
            list_of_expected_date = []
            list_of_expected_movement = []
            list_of_expected_cities = []
            for tag in data_expected:
                for g, h  in enumerate(tag.find_all('span',{'class','calendar'})):
                    list_of_expected_date.append(h.text[-11:])
                for i, j  in enumerate(tag.find_all('span',{'class','capsule'})):
                    list_of_expected_movement.append(j.text)
                for k, l  in enumerate(tag.find_all('div',{'class','location row js-bubble'})):
                    list_of_expected_cities.append(l.text[:-16])

            # Apending data from list to current dict
            current_dict = {}
            for m, milestone in enumerate(sorted(list_of_milestone_date)):
                if list_of_milestone_cities[m] == headline[2].text[:-5] or list_of_milestone_cities[m] == headline[3].text[:-5]:
                    case_milestone = {list_of_milestone_movement[m] +' '+ list_of_milestone_cities[m] :list_of_milestone_date[m]}
                    current_dict.update(case_milestone)
                  
            for e, expected in enumerate(sorted(list_of_expected_date)):
                if list_of_expected_cities[e] == headline[3].text[:-5] or list_of_expected_cities[e] == headline[2].text[:-5]:
                    case_expected = {"EXPECTED " + list_of_expected_movement[e] + ' ' +  list_of_expected_cities[e]:list_of_expected_date[e]}
                    current_dict.update(case_expected)

            # updating current dict with liners, BL number, Ctr Number, POL, POD
            current_dict_fix = dict(sorted(current_dict.items(), key=lambda item: parse_date(item[1])))
            current_dict_fix.update({"Liners":"CNC"})
            current_dict_fix.update({"BL Number":bls})
            current_dict_fix.update({"Container Number":headline[0].text})
            current_dict_fix.update({"POL":headline[2].text[:-5]})
            current_dict_fix.update({"POD":headline[3].text[:-5]})

            # appending to list of dict
            hasil_akhir.append(current_dict_fix)
            print(bls, headline[0].text, " DONE")
            time.sleep(1)

    except Exception as e:
        # for failed bl
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)

  0%|          | 0/30 [00:00<?, ?it/s]

ARM0300221 consist of 1 containers
ARM0300221 CMAU6803365  DONE


  3%|▎         | 1/30 [00:14<06:54, 14.31s/it]

ARM0301958 consist of 1 containers
ARM0301958 CMAU6906471  DONE


  7%|▋         | 2/30 [00:31<07:27, 15.97s/it]

...

 93%|█████████▎| 28/30 [08:18<00:34, 17.35s/it]

ARM0303661 consist of  2  containers
ARM0303661 ECMU9920352  DONE
ARM0303661 CMAU8551684  DONE


 97%|█████████▋| 29/30 [08:34<00:16, 16.83s/it]

ARM0303044 consist of  3  containers
ARM0303044 SEKU4531159  DONE
ARM0303044 CMAU6899794  DONE
ARM0303044 CMAU6177750  DONE


100%|██████████| 30/30 [08:49<00:00, 17.66s/it]


In [8]:
gagal

[]

In [9]:
hasil_akhir2 = hasil_akhir

In [10]:
hasil_akhir2

[{'Empty to shipper JAKARTA': '17-JUN-2023',
  'Ready to be loaded JAKARTA': '19-JUN-2023',
  'Loaded on board JAKARTA': '25-JUN-2023',
  'Vessel Departure JAKARTA': '26-JUN-2023',
  'Vessel Arrival CAGAYAN DE ORO': '21-JUL-2023',
  'Discharged CAGAYAN DE ORO': '21-JUL-2023',
  'Liners': 'CNC',
  'BL Number': 'ARM0300221',
  'Container Number': 'CMAU6803365',
  'POL': 'JAKARTA',
  'POD': 'CAGAYAN DE ORO'},
 {'Empty to shipper JAKARTA': '05-JUL-2023',
  'Ready to be loaded JAKARTA': '08-JUL-2023',
  'Loaded on board JAKARTA': '09-JUL-2023',
  'Vessel Departure JAKARTA': '09-JUL-2023',
  'Vessel Arrival CAGAYAN DE ORO': '04-AUG-2023',
  'Discharged CAGAYAN DE ORO': '04-AUG-2023',
  'Liners': 'CNC',
  'BL Number': 'ARM0301958',
  'Container Number': 'CMAU6906471',
  'POL': 'JAKARTA',
  'POD': 'CAGAYAN DE ORO'},
 {'Empty to shipper JAKARTA': '22-JUL-2023',
  'Ready to be loaded JAKARTA': '25-JUL-2023',
  'Loaded on board JAKARTA': '31-JUL-2023',
  'Vessel Departure JAKARTA': '31-JUL-2023',

In [11]:
# changeing city name in milestone to origin and destination refering from POL and POD
list_of_dict_fix = []
for fd, filter_dict in enumerate(hasil_akhir2):
    try:
        print(filter_dict["Container Number"], fd)
        replacement_mapping = {
            filter_dict["POL"]: 'Origin',
            filter_dict["POD"]: 'Destination'
        }
    except Exception as e:
        print(e)
        gagal.append(filter_dict["BL Number"])
        

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            try:
                key = key.replace(old_key, new_key)
            except:
                pass
        updated_dict[key] = value

    list_of_dict_fix.append(updated_dict)

CMAU6803365 0
CMAU6906471 1
TRHU5686314 2
CMAU8936855 3
GCXU5314769 4
SEKU5696322 5
CMAU8718766 6
TCKU6354975 7
TRHU8979018 8
TGBU6789859 9
TCLU1552105 10
BEAU4671679 11
TCNU7566931 12
TGBU4316456 13
CMAU9318698 14
CMAU6651452 15
CMAU6435279 16
SEKU5904642 17
CAAU5965598 18
TEMU8276773 19
TCKU6322237 20
FFAU1610485 21
TGBU6979463 22
CMAU6338556 23
CMAU7665152 24
CMAU9253935 25
CMAU6859390 26
TLLU5034709 27
SEKU5669902 28
CMAU7526389 29
TCNU4314664 30
TCLU9821197 31
CMAU4525033 32
CMAU6879000 33
CMAU7497779 34
BHCU4961587 35
CMAU6609468 36
TRHU5006700 37
CMAU8852498 38
SEGU6359023 39
SEKU5761433 40
TXGU7146680 41
CMAU8827659 42
CMAU8535106 43
TCKU6232468 44
SEKU5921803 45
TLLU4797120 46
FFAU2162030 47
TRHU6499283 48
TLLU4927461 49
CMAU4347395 50
CMAU6839352 51
CMAU7507579 52
CMAU8456183 53
SEKU5881212 54
TCLU6443030 55
CMAU8724877 56
SEKU6076687 57
SEKU6090427 58
TLLU7706696 59
TRHU8958817 60
TRHU6649824 61
TRHU8924150 62
TXGU5269428 63
CMAU8582181 64
CAAU6291833 65
CMAU6227064 66
CAAU5

In [12]:
gagal

[]

In [13]:
# change key to git db format
list_of_dict_fix2 = []
for filter_dict in list_of_dict_fix:
    replacement_mapping = {
        "POL" : "From",
        "POD" : "To",
        "Vessel Departure Origin": 'ATD',
        "Discharged Destination": 'ATA',
        "EXPECTED Vessel Arrival Destination" : "ETD",
        "Container to consignee Destination": 'Container Release',
        "Empty in depot Destination" : 'Container Return'
    }

    updated_dict = {}

    for key, value in filter_dict.items():
        for old_key, new_key in replacement_mapping.items():
            key = key.replace(old_key, new_key)
        updated_dict[key] = value

        if is_date(value):
            input_date = datetime.strptime(value, "%d-%b-%Y")
            updated_dict[key] = input_date.strftime("%Y-%m-%d")


    list_of_dict_fix2.append(updated_dict)

In [14]:
list_of_dict_fix2[0]

{'Empty to shipper Origin': '2023-06-17',
 'Ready to be loaded Origin': '2023-06-19',
 'Loaded on board Origin': '2023-06-25',
 'ATD': '2023-06-26',
 'Vessel Arrival Destination': '2023-07-21',
 'ATA': '2023-07-21',
 'Liners': 'CNC',
 'BL Number': 'ARM0300221',
 'Container Number': 'CMAU6803365',
 'From': 'JAKARTA',
 'To': 'CAGAYAN DE ORO'}

In [16]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(list_of_dict_fix2)

Today's Collection Name ===>  all_tracking_Sep-18-2023
Today's Collection Has Been Made
Inserting Many Complete!!


In [26]:
# exporting to excel
df = pd.DataFrame(list_of_dict_fix2)
excel_file_path = 'export excel/CNC.xlsx'
df.to_excel(excel_file_path, index=False)